In [1]:
import numpy as np
from numpy import linalg as LA
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal, fft
import math as math
import os
from matplotlib.patches import Polygon
import copy

In [2]:
import pickle
a_file = open("HCPS.pkl", "rb") #remember to close afterwards
HCPS = pickle.load(a_file)
a_file.close()

b_file = open("DATA.pkl", "rb") #remember to close afterwards
DATA = pickle.load(b_file)
b_file.close()

c_file = open("SIM.pkl", "rb") #remember to close afterwards
SIM = pickle.load(c_file)
c_file.close()


HCPS.keys()

dict_keys(['subject0', 'subject1', 'subject2', 'subject3', 'subject4', 'subject5', 'subject6', 'subject7', 'subject8', 'subject9', 'subject10', 'subject11', 'subject12', 'subject13', 'subject14'])

In [3]:
M = lambda s : 1/(s**2 + s) #2nd order machine

num_cond = 7 #number of conditions
participants_num = len(HCPS.keys())

#number of trials for each condition
trialnum = [0] * num_cond 
for c in range(num_cond):
  trialnum[c] = len(HCPS['subject0']['condition'+str(c)].keys())         #number of data trials, trial0 ~ trial3

#parameters (same for all three conditions)
fs = 60                               #pygame update rate 60 Hz
base_freq = 0.05                      #1/20 Hz
N = len(HCPS['subject0']['condition0']['trial0']['time_'])    #data length
xf_all = fft.fftfreq(N, 1./ fs)       #freq (x-axis) both + and - terms
xf = fft.fftfreq(N, 1./ fs)[:N//2]    #freq (x-axis) positive-frequency terms
M_h = M(1.j*2*np.pi*xf_all)           #M_hat = 1/ ((jw)^2 + (jw))
t = HCPS['subject0']['condition0']['trial0']['time_']         #time
prime = np.asarray([2, 3, 5, 7, 
                    11, 13, 17, 19])  #prime numbers
stimulated_index = prime*2 #array([ 4,  6, 10, 14, 22, 26, 34, 38])
stimulated_freq = prime*base_freq

#scaling factors for output screen and input slider
scaleOutputScreen = 1/4
scaleInput = 0.04616974606700115

C:\Users\hachou\AppData\Local\Temp/ipykernel_14356/4009719490.py:1: RuntimeWarning: divide by zero encountered in true_divide
  M = lambda s : 1/(s**2 + s) #2nd order machine
C:\Users\hachou\AppData\Local\Temp/ipykernel_14356/4009719490.py:1: RuntimeWarning: invalid value encountered in true_divide
  M = lambda s : 1/(s**2 + s) #2nd order machine


## TUR & TUD

In [4]:
def TUR_TUD(condition,input_type,trialnum): #input_type = 'U', 'U0'(emg in fusion), or 'U1'(slider in fusion)
  #find index of simulated freqs: trial 0 is EO (ref - Even, dis - Odd)
  # Even_index = (np.where(abs(condition['R'][0]) > 1e-12)[0])[:4] #array([ 6, 14, 26, 38])
  # Odd_index = (np.where(abs(condition['D'][0]) > 1e-12)[0])[:4]  #array([ 4, 10, 22, 34])

  Even_index = np.array([ 6, 14, 26, 38])
  Odd_index = np.array([ 4, 10, 22, 34])

  #R,D in EO trials, without zeros
  even_R = condition['R'][0][Even_index] / scaleOutputScreen #even R without zeros
  odd_D = condition['D'][0][Odd_index] / scaleInput #odd D without zeros

  #R,D in OE trials, without zeros
  odd_R = condition['R'][1][Odd_index] / scaleOutputScreen #odd R without zeros
  even_D = condition['D'][1][Even_index] / scaleInput #even D without zeros

  #number of Tur & Tud = half(trials) (1 EO & 1 OE trial together as one)
  Tur = np.zeros((math.ceil(trialnum/2),8), dtype=complex) #number of stimulatd freqs = 8
  Tud = np.zeros((math.ceil(trialnum/2),8), dtype=complex)

  # U at simulated freqs
  even_U = []; odd_U = []
  for i in range(trialnum): #all trials in a condition
    even_U.append( condition[input_type][i][Even_index] / scaleInput )
    odd_U.append( condition[input_type][i][Odd_index] / scaleInput )
  
  #Lists of Even (EO) and Odd (OE) trials
  EO = [x for x in range(trialnum) if x % 2 == 0] # EO trials (ref - Even, dis - Odd)
  OE = [x for x in range(trialnum) if x % 2 != 0]  # OE trials (ref - Odd, dis - Even)

  # Tur & Tud at simulated freqs
  for i in EO: #EO trials = [0,2,4 ...] (ref - Even, dis - Odd) (dis-ref-dis ...)
    Tur[i//2][1::2] = even_U[i] / even_R 
    Tud[i//2][::2] = odd_U[i] / odd_D 
  for i in OE: #OE trials = [1,3,5 ...] (ref - Odd, dis - Even) (ref-dis-ref...)
    Tur[i//2][::2] = odd_U[i] / odd_R 
    Tud[i//2][1::2] = even_U[i] / even_D 

  return Tur, Tud

In [5]:
#create new nested dic "TUR" "TUD"
TUR = {}; TUD = {}
for p in range(participants_num): # number of participants = 15
    TUR['subject'+str(p)] = {}
    TUD['subject'+str(p)] = {}
    for c in range(num_cond):  # number of conditions  = 7
        TUR['subject'+str(p)]['condition'+str(c)], TUD['subject'+str(p)]['condition'+str(c)] = TUR_TUD(DATA['FREQ']['subject'+str(p)]['condition'+str(c)],'U',trialnum[c])
#create new nested dic "TU0R" "TU0D"
TU0R = {}; TU0D = {}
for p in range(participants_num): # number of participants = 15
    TU0R['subject'+str(p)] = {}
    TU0D['subject'+str(p)] = {}
    for c in range(num_cond):  # number of conditions  = 7
        TU0R['subject'+str(p)]['condition'+str(c)], TU0D['subject'+str(p)]['condition'+str(c)] = TUR_TUD(DATA['FREQ']['subject'+str(p)]['condition'+str(c)],'U0',trialnum[c])
#create new nested dic "TU1R" "TU1D"
TU1R = {}; TU1D = {}
for p in range(participants_num): # number of participants = 15
    TU1R['subject'+str(p)] = {}
    TU1D['subject'+str(p)] = {}
    for c in range(num_cond):  # number of conditions  = 7
        TU1R['subject'+str(p)]['condition'+str(c)], TU1D['subject'+str(p)]['condition'+str(c)] = TUR_TUD(DATA['FREQ']['subject'+str(p)]['condition'+str(c)],'U1',trialnum[c])

## TYR & TYD

In [6]:
def TYR_TYD(condition,trialnum):

  #find index of simulated freqs: trial 0 is EO (ref - Even, dis - Odd)
  # Even_index = (np.where(abs(condition['R'][0]) > 1e-12)[0])[:4] #array([ 6, 14, 26, 38])
  # Odd_index = (np.where(abs(condition['D'][0]) > 1e-12)[0])[:4]  #array([ 4, 10, 22, 34])
  Even_index = np.array([ 6, 14, 26, 38])
  Odd_index = np.array([ 4, 10, 22, 34])

  #R,D in EO trials, without zeros
  even_R = condition['R'][0][Even_index] / scaleOutputScreen #even R without zeros
  odd_D = condition['D'][0][Odd_index] / scaleInput #odd D without zeros

  #R,D in OE trials, without zeros
  odd_R = condition['R'][1][Odd_index] / scaleOutputScreen #odd R without zeros
  even_D = condition['D'][1][Even_index] / scaleInput #even D without zeros

  #number of Tyr & Tyd = half(trials) (1 EO & 1 OE trial together as one)
  Tyr = np.zeros((math.ceil(trialnum/2),8), dtype=complex) #number of stimulatd freqs = 8
  Tyd = np.zeros((math.ceil(trialnum/2),8), dtype=complex)

  # Y at simulated freqs
  even_Y = []; odd_Y = []
  for i in range(trialnum): #all trials in a condition
    even_Y.append( condition['Y'][i][Even_index] / scaleOutputScreen )
    odd_Y.append( condition['Y'][i][Odd_index] / scaleOutputScreen )

  #Lists of Even (EO) and Odd (OE) trials
  EO = [x for x in range(trialnum) if x % 2 == 0] # EO trials (ref - Even, dis - Odd)
  OE = [x for x in range(trialnum) if x % 2 != 0]  # OE trials (ref - Odd, dis - Even)

  # Tyr & Tyd at simulated freqs
  for i in EO: #EO trials = [0,2,4 ...] (ref - Even, dis - Odd) (dis-ref-dis ...)
    Tyr[i//2][1::2] = even_Y[i] / even_R 
    Tyd[i//2][::2] = odd_Y[i] / odd_D 
  for i in OE: #OE trials = [1,3,5 ...] (ref - Odd, dis - Even) (ref-dis-ref...)
    Tyr[i//2][::2] = odd_Y[i] / odd_R 
    Tyd[i//2][1::2] = even_Y[i] / even_D 

  return Tyr, Tyd

In [7]:
#create new nested dic "TYR" "TYD"
TYR = {}; TYD = {}
for p in range(participants_num): # number of participants = 15
    TYR['subject'+str(p)] = {}
    TYD['subject'+str(p)] = {}
    for c in range(num_cond):  # number of conditions  = 7
        TYR['subject'+str(p)]['condition'+str(c)], TYD['subject'+str(p)]['condition'+str(c)] = TYR_TYD(DATA['FREQ']['subject'+str(p)]['condition'+str(c)],
                                                                                                       trialnum[c])

## TyMd

In [8]:
def TYMD_fun(condition,trialnum):

  #find index of simulated freqs: trial 0 is EO (ref - Even, dis - Odd)
  # Even_index = (np.where(abs(condition['R'][0]) > 1e-12)[0])[:4] #array([ 6, 14, 26, 38])
  # Odd_index = (np.where(abs(condition['D'][0]) > 1e-12)[0])[:4]  #array([ 4, 10, 22, 34])
  Even_index = np.array([ 6, 14, 26, 38])
  Odd_index = np.array([ 4, 10, 22, 34])

  #R,D in EO trials, without zeros
  even_R = condition['R'][0][Even_index] / scaleOutputScreen #even R without zeros
  odd_D = condition['MD'][0][Odd_index] / scaleOutputScreen #odd D without zeros

  #R,D in OE trials, without zeros
  odd_R = condition['R'][1][Odd_index] / scaleOutputScreen #odd R without zeros
  even_D = condition['MD'][1][Even_index] / scaleOutputScreen #even D without zeros

  #number of Tyr & Tyd = half(trials) (1 EO & 1 OE trial together as one)
  Tyr = np.zeros((math.ceil(trialnum/2),8), dtype=complex) #number of stimulatd freqs = 8
  Tymd = np.zeros((math.ceil(trialnum/2),8), dtype=complex)

  # Y at simulated freqs
  even_Y = []; odd_Y = []
  for i in range(trialnum): #all trials in a condition
    even_Y.append( condition['Y'][i][Even_index] / scaleOutputScreen )
    odd_Y.append( condition['Y'][i][Odd_index] / scaleOutputScreen )

  #Lists of Even (EO) and Odd (OE) trials
  EO = [x for x in range(trialnum) if x % 2 == 0] # EO trials (ref - Even, dis - Odd)
  OE = [x for x in range(trialnum) if x % 2 != 0]  # OE trials (ref - Odd, dis - Even)

  # Tyr & Tyd at simulated freqs
  for i in EO: #EO trials = [0,2,4 ...] (ref - Even, dis - Odd)
    Tyr[i//2][1::2] = even_Y[i] / even_R 
    Tymd[i//2][::2] = odd_Y[i] / odd_D 
  for i in OE: #OE trials = [1,3,5 ...] (ref - Odd, dis - Even)
    Tyr[i//2][::2] = odd_Y[i] / odd_R 
    Tymd[i//2][1::2] = even_Y[i] / even_D 

  return Tymd

In [9]:
#create new nested dic "TYR" "TYD"
TYMD = {}
for p in range(participants_num): # number of participants = 15
    TYMD['subject'+str(p)] = {}
    for c in range(num_cond):  # number of conditions  = 7
        TYMD['subject'+str(p)]['condition'+str(c)] = TYMD_fun(DATA['FREQ']['subject'+str(p)]['condition'+str(c)],trialnum[c])

## simulated TY0R TY0D & TY1R TY1D for condition 2-4

In [10]:
def SIM_TYR_TYD(condition,sim_condition,y_type,trialnum):

  #find index of simulated freqs: trial 0 is EO (ref - Even, dis - Odd)
  # Even_index = (np.where(abs(condition['R'][0]) > 1e-12)[0])[:4] #array([ 6, 14, 26, 38])
  # Odd_index = (np.where(abs(condition['D'][0]) > 1e-12)[0])[:4]  #array([ 4, 10, 22, 34])
  Even_index = np.array([ 6, 14, 26, 38])
  Odd_index = np.array([ 4, 10, 22, 34])

  #R,D in EO trials, without zeros
  even_R = condition['R'][0][Even_index] / scaleOutputScreen #even R without zeros
  odd_D = condition['D'][0][Odd_index] / scaleInput #odd D without zeros

  #R,D in OE trials, without zeros
  odd_R = condition['R'][1][Odd_index] / scaleOutputScreen #odd R without zeros
  even_D = condition['D'][1][Even_index] / scaleInput #even D without zeros

  #number of Tyr & Tyd = half(trials) (1 EO & 1 OE trial together as one)
  Tyr = np.zeros((math.ceil(trialnum/2),8), dtype=complex) #number of stimulatd freqs = 8
  Tyd = np.zeros((math.ceil(trialnum/2),8), dtype=complex)

  # Y at simulated freqs
  even_Y = []; odd_Y = []
  for i in range(trialnum): #all trials in a condition
    even_Y.append( sim_condition[y_type][i][Even_index] / scaleOutputScreen ) #Y0 and Y1 are from numerical_sim
    odd_Y.append( sim_condition[y_type][i][Odd_index] / scaleOutputScreen )

  #Lists of Even (EO) and Odd (OE) trials
  EO = [x for x in range(trialnum) if x % 2 == 0] # EO trials (ref - Even, dis - Odd)
  OE = [x for x in range(trialnum) if x % 2 != 0]  # OE trials (ref - Odd, dis - Even)

  # Tyr & Tyd at simulated freqs
  for i in EO: #EO trials = [0,2,4 ...] (ref - Even, dis - Odd)
    Tyr[i//2][1::2] = even_Y[i] / even_R 
    Tyd[i//2][::2] = odd_Y[i] / odd_D 
  for i in OE: #OE trials = [1,3,5 ...] (ref - Odd, dis - Even)
    Tyr[i//2][::2] = odd_Y[i] / odd_R 
    Tyd[i//2][1::2] = even_Y[i] / even_D 

  return Tyr, Tyd

In [11]:
#create new nested dic "TY0R" "TY0D"
TY0R = {}; TY0D = {}
for p in range(participants_num): # number of participants = 15
    TY0R['subject'+str(p)] = {}
    TY0D['subject'+str(p)] = {}
    for c in [2,3,4]:  # fusion conditions
        TY0R['subject'+str(p)]['condition'+str(c)], TY0D['subject'+str(p)]['condition'+str(c)] = SIM_TYR_TYD(DATA['FREQ']['subject'+str(p)]['condition'+str(c)],
                                                                                                       SIM['subject'+str(p)]['condition'+str(c)],'Y0',
                                                                                                       trialnum[c])

TY1R = {}; TY1D = {}
for p in range(participants_num): # number of participants = 15
    TY1R['subject'+str(p)] = {}
    TY1D['subject'+str(p)] = {}
    for c in [2,3,4]:  # fusion conditions
        TY1R['subject'+str(p)]['condition'+str(c)], TY1D['subject'+str(p)]['condition'+str(c)] = SIM_TYR_TYD(DATA['FREQ']['subject'+str(p)]['condition'+str(c)],
                                                                                                       SIM['subject'+str(p)]['condition'+str(c)],'Y1',
                                                                                                       trialnum[c])                                                                                                

## F & B

In [12]:
def F_B(Tur,Tud,trialnum):
  F = np.zeros((math.ceil(trialnum/2),8), dtype=complex) #number of stimulated freqs = 8
  B = np.zeros((math.ceil(trialnum/2),8), dtype=complex)
  for i in range(math.ceil(trialnum/2)):
    for w in range(8): #number of stimulated freq
      B[i][w] = - (1 / M_h[stimulated_index][w]) * (Tud[i][w] / (1+ Tud[i][w])) 
      F[i][w] = (1 + B[i][w] * M_h[stimulated_index][w])*Tur[i][w] - B[i][w]
  return F,B

# def F_B(Tur,Tud,trialnum):
#   F = np.zeros((math.ceil(trialnum/2),8), dtype=complex) #number of stimulated freqs = 8
#   B = np.zeros((math.ceil(trialnum/2),8), dtype=complex)
#   for i in range(math.ceil(trialnum/2)):
#     B[i] = - (1 / M_h[stimulated_index]) * (Tud[i] / (1+ Tud[i])) 
#     F[i] = (1 + B[i] * M_h[stimulated_index])*Tur[i] - B[i]
#   return F,B

In [13]:
F = {}; B = {}
for p in range(participants_num): # number of participants = 15
    F['subject'+str(p)] = {}
    B['subject'+str(p)] = {}
    for c in range(num_cond):  # number of conditions  = 7
        F['subject'+str(p)]['condition'+str(c)], B['subject'+str(p)]['condition'+str(c)] = F_B(TUR['subject'+str(p)]['condition'+str(c)],
                                                                                               TUD['subject'+str(p)]['condition'+str(c)],
                                                                                               trialnum[c])
F0 = {}; B0 = {}
for p in range(participants_num): # number of participants = 15
    F0['subject'+str(p)] = {}
    B0['subject'+str(p)] = {}
    for c in range(num_cond):  # number of conditions  = 7
        F0['subject'+str(p)]['condition'+str(c)], B0['subject'+str(p)]['condition'+str(c)] = F_B(TU0R['subject'+str(p)]['condition'+str(c)],
                                                                                               TU0D['subject'+str(p)]['condition'+str(c)],
                                                                                               trialnum[c])
F1 = {}; B1 = {}
for p in range(participants_num): # number of participants = 15
    F1['subject'+str(p)] = {}
    B1['subject'+str(p)] = {}
    for c in range(num_cond):  # number of conditions  = 7
        F1['subject'+str(p)]['condition'+str(c)], B1['subject'+str(p)]['condition'+str(c)] = F_B(TU1R['subject'+str(p)]['condition'+str(c)],
                                                                                               TU1D['subject'+str(p)]['condition'+str(c)],
                                                                                               trialnum[c])

## find crossover freq

In [14]:
# gain of open loop transfer function L = BM at the stim freqs, find L of all trials all participants
L = []
for p in range(participants_num):
    for c in range(num_cond):   
        for i in range(trialnum[c]//2):
            # OL_TF = (M_h[stimulated_index] * B['subject'+str(p)]['condition'+str(c)][i]) / (1+M_h[stimulated_index] * B['subject'+str(p)]['condition'+str(c)][i])
            OL_TF = (M_h[stimulated_index] * B['subject'+str(p)]['condition'+str(c)][i])  #L = BM
            L.append( np.abs(OL_TF) )
L_median = np.percentile(L,50,axis = 0)
# L_median = np.median(L,axis =0)
L_median

array([0.82978331, 0.8950932 , 0.70060568, 0.65458824, 0.3504692 ,
       0.28882735, 0.21209986, 0.17936504])

## crossover freq in differnet conditions

In [15]:
# gain of open loop transfer function L = BM at the stim freqs, find L of all trials all participants
L = []
for p in range(participants_num):
    for c in [5]:   
        for i in range(trialnum[c]//2):
            L.append( np.abs(B['subject'+str(p)]['condition'+str(c)][i] * M_h[stimulated_index]) )
# L_mean_slider = np.percentile(L,50,axis = 0)
L_mean_slider = np.mean(L,axis = 0)
L_mean_slider

array([8.81878266, 1.40367129, 0.92056369, 0.70904003, 0.38855738,
       0.27936669, 0.21940464, 0.19648346])

In [16]:
# gain of open loop transfer function L = BM at the stim freqs, find L of all trials all participants
L = []
for p in range(participants_num):
    for c in [1,6]:   
        for i in range(trialnum[c]//2):
            L.append( np.abs(B['subject'+str(p)]['condition'+str(c)][i] * M_h[stimulated_index]) )
# L_mean_emg = np.percentile(L,50,axis = 0)
L_mean_emg = np.mean(L,axis = 0)
L_mean_emg

array([1.65344472, 1.57835112, 0.83532582, 0.64944083, 0.45512487,
       0.66525288, 0.43512372, 0.33557466])

In [17]:
# gain of open loop transfer function L = BM at the stim freqs, find L of all trials all participants
L = []
for p in range(participants_num):
    for c in [2,3,4]:   
        for i in range(trialnum[c]//2):
            L.append( np.abs(B['subject'+str(p)]['condition'+str(c)][i] * M_h[stimulated_index]) )
# L_mean = np.percentile(L,50,axis = 0)
L_mean = np.mean(L,axis = 0)
L_mean

array([1.50161782, 1.44157609, 0.79533432, 0.74063069, 0.38612708,
       0.3916075 , 0.28172336, 0.26512993])

In [18]:
# gain of open loop transfer function L = BM at the stim freqs, find L of all trials all participants
L0 = []
for p in range(participants_num):
    for c in [2,3,4]:   
        for i in range(trialnum[c]//2):
            L0.append( np.abs(B0['subject'+str(p)]['condition'+str(c)][i] * M_h[stimulated_index]) )
L0_mean = np.mean(L0,axis = 0)
L0_mean

array([1.0652558 , 1.05468175, 0.80725566, 0.6588671 , 0.44908044,
       0.54608752, 0.37416402, 0.44450172])

In [19]:
L1 = []
for p in range(participants_num):
    for c in [2,3,4]: 
        for i in range(trialnum[c]//2):
            L1.append( np.abs(B1['subject'+str(p)]['condition'+str(c)][i] * M_h[stimulated_index]) )
L1_mean = np.mean(L1,axis = 0)
L1_mean

array([1.29415497, 1.00242813, 0.68740336, 0.81972949, 0.34176445,
       0.26712171, 0.17083459, 0.15135591])

# cerate dicts

In [20]:
#create nested dict
TF = {  'TUR': TUR, 
        'TUD': TUD,
        'TU0R': TU0R,
        'TU0D': TU0D,
        'TU1R': TU1R, 
        'TU1D': TU1D,
        'TYR': TYR,
        'TYD': TYD, 
        'TYMD': TYMD,
        'TY0R': TY0R,
        'TY0D': TY0D,
        'TY1R': TY1R,
        'TY1D': TY1D,
        'F': F,
        'B': B,
        'F0': F0,
        'B0': B0,
        'F1': F1,
        'B1': B1}

In [21]:
c_file = open("TF.pkl", "wb")
pickle.dump(TF, c_file)
c_file.close()